In [2]:
from pyspark.sql import SQLContext 

sqlContext = SQLContext(sc) 
df = sqlContext.read.format('com.databricks.spark.xml').options(rowTag='page').load('s3a://cap4770-2024summer-kl/example.xml')
df.rdd.collect()

/home/ubuntu/spark-3.5.1-bin-hadoop3/python/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


[Row(revision=Row(text=' no outlinks here '), title='A'),
 Row(revision=Row(text=' link to [[F]] and link to [[A]] '), title='B'),
 Row(revision=Row(text=' link to [[A]] and link to [[D]] '), title='C'),
 Row(revision=Row(text=' no outlinks here (no inlinks to E either) '), title='E'),
 Row(revision=Row(text=' no outlinks here '), title='F')]

In [20]:
import re 

def get_links(text, title_list): 
    pattern = re.compile(r"\[\[(.*?)\]\]") 
    res = pattern.findall(text) 
    links = [links.split('|')[0] for links in res] if res != None else []
    if links:
        links = [link.replace(' ', '_') for link in links if link in title_list]
    return links

In [4]:
titles = df.rdd.map(lambda x: x['title'])
titles.collect()

['A', 'B', 'C', 'E', 'F']

In [5]:
title_list = titles.collect()
links = df.rdd.map(lambda x: get_links(x['revision']['text'], title_list))
links.collect()

[[], ['F', 'A'], ['A'], [], []]

In [6]:
flat_links = {link for link_list in links.collect() for link in link_list}

In [7]:
valid_links = set(titles.collect()).intersection(flat_links)
valid_links

{'A', 'F'}

In [9]:
adj_graph = titles.zip(links)
adj_graph.collect()

[('A', []), ('B', ['F', 'A']), ('C', ['A']), ('E', []), ('F', [])]

In [10]:
df = sqlContext.read.format('com.databricks.spark.xml').options(rowTag='page').load('s3a://cap4770-2024summer-kl/enwiki-latest-pages-articles1-trunc.xml')

In [21]:
wiki_titles = df.rdd.map(lambda x: x['title'])

In [22]:
wiki_titles_list = wiki_titles.collect()
wiki_titles = wiki_titles.map(lambda x: x.replace(' ', '_'))
wiki_links = df.rdd.map(lambda x: get_links(x['revision']['text'], wiki_titles_list))

In [23]:
wiki_adj_graph = wiki_titles.zip(wiki_links)

In [24]:
wiki_adj_graph.saveAsTextFile('s3a://cap4770-2024summer-kl/adj.txt')